##### SOLUTION

### Study session 11 - SQL

#### BIOINF 575 - Fall 2023



In [1]:
# put the import statements at the top of the notebook

# bring in the functionality from the sqlite3 module
# specifically the connect function

from sqlite3 import connect

# import pandas so you can see the select results as a dataframe
import pandas as pd

___

<b> <font color = "red">Exercise</font></b>

#### Create a database for gene annotations (gene ontology - GO -  terms annotations)

Create a SQLite database with a genes table, a GO_terms table, and a gene_GO_term table.   
The genes table should have the gene id, gene symbol and the gene description.     
The GO_terms table should have the GO term id and the GO term name/description.     
The gene to GO term table should have the gene id and the GO term id.     


In [2]:
# Write your solution here

# Write the create table statements here - all strings

#-------
create_genes = '''
CREATE TABLE IF NOT EXISTS genes (
      gene_id INTEGER PRIMARY KEY AUTOINCREMENT,
      gene_symbol TEXT NOT NULL,
      gene_description TEXT NOT NULL
    );
'''
#-------

create_GO_terms = '''
CREATE TABLE IF NOT EXISTS GO_terms (
      GO_term_id INTEGER PRIMARY KEY AUTOINCREMENT,
      name TEXT NOT NULL
    );
'''
#-------

create_gene_GO_term = '''
CREATE TABLE IF NOT EXISTS gene_GO_term (
      gene_id INTEGER NOT NULL,                             -- REFERENCES  gene_id in the genes table
      GO_term_id INTEGER NOT NULL,                          -- REFERENCES  GO_term_id in the GO_terms table
      
      PRIMARY KEY (gene_id, GO_term_id),                    -- we set up a multi-column primary key - table constraint
                                                                -- the two columns uniquely indetify a row 
                                                                -- this will automatically create an index in the database
      FOREIGN KEY (gene_id) REFERENCES  genes  (gene_id),
      FOREIGN KEY (GO_term_id) REFERENCES  GO_terms  (GO_term_id)
    );
'''
#-------

In [3]:
# we do the same thing multiple times - we can use a function

def run_create_table(sql, cursor, connection):
    try:
        cursor.execute(sql)
    except connection.DatabaseError:
        print("Creating the table resulted in a database error!")
        connection.rollback()
        raise
    else:
        connection.commit()
    finally:
        print("done!")

In [4]:
# connect to a database ... if the file does not exist it will be created
connection = connect("gene_annotations.sqlite")
cursor = connection.cursor()

In [5]:
# check the sqlite_master table - should be empty at this point

select_master = "SELECT name, type FROM sqlite_master;"
cursor.execute(select_master)
cursor.fetchall()

[]

In [6]:
# run the commands
run_create_table(create_genes, cursor, connection)
run_create_table(create_GO_terms, cursor, connection)
run_create_table(create_gene_GO_term, cursor, connection)

done!
done!
done!


In [7]:
# check the sqlite_master table

# select_master = "SELECT name, type FROM sqlite_master;"
cursor.execute(select_master)
cursor.fetchall()


[('genes', 'table'),
 ('sqlite_sequence', 'table'),
 ('GO_terms', 'table'),
 ('gene_GO_term', 'table'),
 ('sqlite_autoindex_gene_GO_term_1', 'index')]

___

<b> <font color = "red">Exercise</font></b>

#### Populate a database 

Insert data into tables in the GO annotations database you just created.   
Use the data from the file `genes_info1.txt`.


In [8]:
# Write your solution here

# Write the insert statements here - all strings
# because we insert values in all columns we do not need to specify the columns

insert_gene = "INSERT INTO genes VALUES(?,?,?);"
insert_GO_term = "INSERT INTO GO_terms VALUES(?,?);"
insert_gene_GO_term = "INSERT INTO gene_GO_term VALUES(?,?);"


In [9]:
file_name = "genes_info1.txt"

In [10]:
GO_terms_dict = {}
GO_term_index = 0

with open(file_name) as genes_info:
    # read the header line and ignore it
    header_line = genes_info.readline() 
    
    # read the rest of the lines and process them
    for line in genes_info: 
        # break the line by tab and take the information needed in the table
        line_list = line.strip().split("\t")
        
        # we put the gene id in a variable because we need it in gene_GO_term table
        gene_id = line_list[0]
        
        # we take the gene id, symbol, description, 
        # which are the first 3 elements in the list - see header_line to confirm
        # this will be a row in the genes table
        gene_row = line_list[:3]
        cursor.execute(insert_gene, gene_row)
        
        # we take the go terms which are last in the list
        # we breake them up into a list - they are separated by ";"
        GO_terms_list = line_list[-1].split(";")
        
        # we go through the list of go terms to create rows for the GO_terms table
        # we need to keep track of the GO terms so we do not add them multiple times in the table
        # one GO term can be associated with multiple genes and one genes can be associated with multiple GO terms
        # a dictionary has unique keys and can store values foe each key
        # we will use that to keep track of our GO terms
        for GO_term in GO_terms_list:
            # remove whitespace from the beginning and end of the GO_term
            GO_term_name = GO_term.strip()
            
            # if we did not see this GO term before we add it to the dictionary and create an ID for it
            # otherwise we get the id of the term from the dictionary
            if GO_term_name not in GO_terms_dict:
                GO_term_index += 1
                GO_term_id = GO_term_index
                GO_terms_dict[GO_term_name] = GO_term_index
                # we will add this new GO term data to the GO_term table so prepare a row
                GO_term_row = (GO_term_id, GO_term_name)
                cursor.execute(insert_GO_term, GO_term_row)

            else:
                GO_term_id = GO_terms_dict[GO_term_name]
            
            # we add the connection between the gene and the GO term to the gene_GO_term table
            # we prepare a row
            gene_GO_row = (gene_id, GO_term_id)
            cursor.execute(insert_gene_GO_term, gene_GO_row)
       

In [11]:
# we check the tables and if all is well, we can commit these changes

select_genes = "SELECT * FROM genes;"
select_GO_terms = "SELECT * FROM GO_terms;"
select_gene_GO = "SELECT * FROM gene_GO_term;"


In [12]:
cursor.execute(select_genes)
cursor.fetchall()

[(672, 'BRCA1', 'BRCA1 DNA repair associated'),
 (675, 'BRCA2', 'BRCA2 DNA repair associated'),
 (7040, 'TGFB1', 'transforming growth factor beta 1'),
 (7157, 'TP53', 'tumor protein p53'),
 (10296, 'MAEA', 'macrophage erythroblast attacher, E3 ubiquitin ligase'),
 (64926, 'RASAL3', 'RAS protein activator like 3')]

In [13]:
cursor.execute(select_GO_terms)
cursor.fetchall()

[(1, 'intrinsic apoptotic signaling pathway in response to DNA damage'),
 (2, 'transcription cis-regulatory region binding'),
 (3, 'transcription coactivator activity'),
 (4, 'ubiquitin-protein transferase activity'),
 (5, 'histone acetyltransferase activity'),
 (6, 'protease binding'),
 (7, 'single-stranded DNA binding'),
 (8, 'protein binding'),
 (9, 'negative regulation of transcription, DNA-templated'),
 (10, 'positive regulation of transcription by RNA polymerase II'),
 (11, 'promoter-specific chromatin binding'),
 (12, 'circadian behavior'),
 (13, 'cell adhesion')]

In [14]:
cursor.execute(select_gene_GO)
cursor.fetchall()

[(672, 1),
 (672, 2),
 (672, 3),
 (672, 4),
 (675, 5),
 (675, 6),
 (675, 7),
 (675, 8),
 (7157, 9),
 (7157, 10),
 (7157, 11),
 (7157, 12),
 (10296, 13),
 (10296, 8),
 (10296, 11),
 (7040, 13),
 (7040, 8),
 (64926, 13),
 (64926, 8),
 (64926, 6)]

In [15]:
# if all is well commit otherwise rollback, correct the issue in the code - rerun the code and if all well commit

# connection.commit()
connection.rollback()

In [16]:
## OPTION 2 - make lists and insert all data at the end

# before we can do that we need to clear the table data if we did not rollback option 1

delete_genes = "DELETE FROM genes;"
delete_GO_terms = "DELETE FROM GO_terms;"
delete_gene_GO = "DELETE FROM gene_GO_term;"

cursor.execute(delete_genes)
cursor.execute(delete_GO_terms)
cursor.execute(delete_gene_GO)



In [17]:
# check all 3 tables are empty

cursor.execute(select_genes)
print(cursor.fetchall())

cursor.execute(select_GO_terms)
print(cursor.fetchall())

cursor.execute(select_gene_GO)
print(cursor.fetchall())



[]
[]
[]


In [18]:
# commit the change

connection.commit()

In [19]:
GO_terms_dict = {}
GO_term_index = 0

genes_rows = []
GO_terms_rows = []
gene_GO_rows = []

with open(file_name) as genes_info:
    # read the header line and ignore it
    header_line = genes_info.readline() 
    
    # read the rest of the lines and process them
    for line in genes_info: 
        # break the line by tab and take the information needed in the table
        line_list = line.strip().split("\t")
        
        # we put the gene id in a variable because we need it in gene_GO_term table
        gene_id = line_list[0]
        
        # we take the gene id, symbol, description, 
        # which are the first 3 elements in the list - see header_line to confirm
        # this will be a row in the genes table
        gene_row = line_list[:3]
        genes_rows.append(gene_row)
        
        # we take the go terms which are last in the list
        # we breake them up into a list - they are separated by ";"
        GO_terms_list = line_list[-1].split(";")
        
        # we go through the list of go terms to create rows for the GO_terms table
        # we need to keep track of the GO terms so we do not add them multiple times in the table
        # one GO term can be associated with multiple genes and one genes can be associated with multiple GO terms
        # a dictionary has unique keys and can store values foe each key
        # we will use that to keep track of our GO terms
        for GO_term in GO_terms_list:
            # remove whitespace from the beginning and end of the GO_term
            GO_term_name = GO_term.strip()
            
            # if we did not see this GO term before we add it to the dictionary and create an ID for it
            # otherwise we get the id of the term from the dictionary
            if GO_term_name not in GO_terms_dict:
                GO_term_index += 1
                GO_term_id = GO_term_index
                GO_terms_dict[GO_term_name] = GO_term_index
                # we will add this new GO term data to the GO_term table so prepare a row
                GO_term_row = (GO_term_id, GO_term_name)
                GO_terms_rows.append(GO_term_row)

            else:
                GO_term_id = GO_terms_dict[GO_term_name]
            
            # we add the connection between the gene and the GO term to the gene_GO_term table
            # we prepare a row
            gene_GO_row = (gene_id, GO_term_id)
            gene_GO_rows.append(gene_GO_row)
       

In [20]:
# check the lists

genes_rows


[['672', 'BRCA1', 'BRCA1 DNA repair associated'],
 ['675', 'BRCA2', 'BRCA2 DNA repair associated'],
 ['7157', 'TP53', 'tumor protein p53'],
 ['10296', 'MAEA', 'macrophage erythroblast attacher, E3 ubiquitin ligase'],
 ['7040', 'TGFB1', 'transforming growth factor beta 1'],
 ['64926', 'RASAL3', 'RAS protein activator like 3']]

In [21]:
GO_terms_rows

[(1, 'intrinsic apoptotic signaling pathway in response to DNA damage'),
 (2, 'transcription cis-regulatory region binding'),
 (3, 'transcription coactivator activity'),
 (4, 'ubiquitin-protein transferase activity'),
 (5, 'histone acetyltransferase activity'),
 (6, 'protease binding'),
 (7, 'single-stranded DNA binding'),
 (8, 'protein binding'),
 (9, 'negative regulation of transcription, DNA-templated'),
 (10, 'positive regulation of transcription by RNA polymerase II'),
 (11, 'promoter-specific chromatin binding'),
 (12, 'circadian behavior'),
 (13, 'cell adhesion')]

In [22]:
gene_GO_rows

[('672', 1),
 ('672', 2),
 ('672', 3),
 ('672', 4),
 ('675', 5),
 ('675', 6),
 ('675', 7),
 ('675', 8),
 ('7157', 9),
 ('7157', 10),
 ('7157', 11),
 ('7157', 12),
 ('10296', 13),
 ('10296', 8),
 ('10296', 11),
 ('7040', 13),
 ('7040', 8),
 ('64926', 13),
 ('64926', 8),
 ('64926', 6)]

In [23]:
# insert the data from the lists 
# because we have many rows in the lists we use execute many

cursor.executemany(insert_gene, genes_rows)

In [24]:
cursor.executemany(insert_GO_term, GO_terms_rows)

In [25]:
cursor.executemany(insert_gene_GO_term, gene_GO_rows)

In [26]:
# check the data

cursor.execute(select_genes)
cursor.fetchall()

[(672, 'BRCA1', 'BRCA1 DNA repair associated'),
 (675, 'BRCA2', 'BRCA2 DNA repair associated'),
 (7040, 'TGFB1', 'transforming growth factor beta 1'),
 (7157, 'TP53', 'tumor protein p53'),
 (10296, 'MAEA', 'macrophage erythroblast attacher, E3 ubiquitin ligase'),
 (64926, 'RASAL3', 'RAS protein activator like 3')]

In [27]:
cursor.execute(select_GO_terms)
cursor.fetchall()

[(1, 'intrinsic apoptotic signaling pathway in response to DNA damage'),
 (2, 'transcription cis-regulatory region binding'),
 (3, 'transcription coactivator activity'),
 (4, 'ubiquitin-protein transferase activity'),
 (5, 'histone acetyltransferase activity'),
 (6, 'protease binding'),
 (7, 'single-stranded DNA binding'),
 (8, 'protein binding'),
 (9, 'negative regulation of transcription, DNA-templated'),
 (10, 'positive regulation of transcription by RNA polymerase II'),
 (11, 'promoter-specific chromatin binding'),
 (12, 'circadian behavior'),
 (13, 'cell adhesion')]

In [28]:
cursor.execute(select_gene_GO)
cursor.fetchall()

[(672, 1),
 (672, 2),
 (672, 3),
 (672, 4),
 (675, 5),
 (675, 6),
 (675, 7),
 (675, 8),
 (7157, 9),
 (7157, 10),
 (7157, 11),
 (7157, 12),
 (10296, 13),
 (10296, 8),
 (10296, 11),
 (7040, 13),
 (7040, 8),
 (64926, 13),
 (64926, 8),
 (64926, 6)]

In [29]:
# commit the changes
connection.commit()

In [30]:
## OPTION 3 - use the autoincrement for the GO_term id since it is not provided to us in the file

# before we can do that we need to clear the table data
delete_genes = "DELETE FROM genes;"
delete_GO_terms = "DELETE FROM GO_terms;"
delete_gene_GO = "DELETE FROM gene_GO_term;"

cursor.execute(delete_genes)
cursor.execute(delete_GO_terms)
cursor.execute(delete_gene_GO)

In [31]:
# commit the change
connection.commit()

In [32]:
# the insert statement for the GO_term table changed since we do not need to provide the id
insert_GO_term1 = "INSERT INTO GO_terms (name) VALUES (?);"


In [33]:
GO_terms_dict = {}

with open(file_name) as genes_info:
    header_line = genes_info.readline() 
    for line in genes_info: 
        line_list = line.strip().split("\t")
        gene_id = line_list[0]

        gene_row = line_list[:3]
        cursor.execute(insert_gene, gene_row)

        GO_terms_list = line_list[-1].split(";")
        
        for GO_term in GO_terms_list:
            GO_term_name = GO_term.strip()
            if GO_term_name not in GO_terms_dict:
                cursor.execute(insert_GO_term1, [GO_term_name])
                
                # get the autogenerated id from the database after the insert
                GO_term_id = cursor.lastrowid
                
                GO_terms_dict[GO_term_name] = GO_term_id

            else:
                GO_term_id = GO_terms_dict[GO_term_name]

            gene_GO_row = (gene_id, GO_term_id)
            cursor.execute(insert_gene_GO_term, gene_GO_row)
       

In [34]:
# check the data

cursor.execute(select_genes)
cursor.fetchall()

[(672, 'BRCA1', 'BRCA1 DNA repair associated'),
 (675, 'BRCA2', 'BRCA2 DNA repair associated'),
 (7040, 'TGFB1', 'transforming growth factor beta 1'),
 (7157, 'TP53', 'tumor protein p53'),
 (10296, 'MAEA', 'macrophage erythroblast attacher, E3 ubiquitin ligase'),
 (64926, 'RASAL3', 'RAS protein activator like 3')]

In [35]:
cursor.execute(select_GO_terms)
cursor.fetchall()

[(14, 'intrinsic apoptotic signaling pathway in response to DNA damage'),
 (15, 'transcription cis-regulatory region binding'),
 (16, 'transcription coactivator activity'),
 (17, 'ubiquitin-protein transferase activity'),
 (18, 'histone acetyltransferase activity'),
 (19, 'protease binding'),
 (20, 'single-stranded DNA binding'),
 (21, 'protein binding'),
 (22, 'negative regulation of transcription, DNA-templated'),
 (23, 'positive regulation of transcription by RNA polymerase II'),
 (24, 'promoter-specific chromatin binding'),
 (25, 'circadian behavior'),
 (26, 'cell adhesion')]

In [36]:
cursor.execute(select_gene_GO)
cursor.fetchall()

[(672, 14),
 (672, 15),
 (672, 16),
 (672, 17),
 (675, 18),
 (675, 19),
 (675, 20),
 (675, 21),
 (7157, 22),
 (7157, 23),
 (7157, 24),
 (7157, 25),
 (10296, 26),
 (10296, 21),
 (10296, 24),
 (7040, 26),
 (7040, 21),
 (64926, 26),
 (64926, 21),
 (64926, 19)]

In [37]:
# commit the changes
connection.commit()

___

<b> <font color = "red">Exercise</font></b>

#### Query a database 

How many genes we have?  
How many go terms we have?   
How many go terms are associated which each gene?   
How many genes are associated with each GO term?   
Select all pairs of gene symbol and go term description that have a record in the gene_GO_term table.

In [38]:
# Write your solution here

# Bring in the functions from class for a nice display of the results

def get_header(cursor):
    '''
    Makes a tab delimited header row from the cursor description.
    Arguments:
        cursor: a cursor after a select query
    Returns:
        string: A string consisting of the column names separated by tabs, no new line
    '''
    return '\t'.join([row[0] for row in cursor.description])

def get_results(cursor):
    '''
    Makes a tab delimited table from the cursor results.
    Arguments:
        cursor: a cursor after a select query
    Returns:
        string: A string consisting of the column names separated by tabs, no new line
    ''' 
    res = list()
    for row in cursor.fetchall():        
        res.append('\t'.join(list(map(str,row))))
    return "\n".join(res)


In [39]:
# query1: How many genes we have?

query1 = "SELECT count(gene_id) 'Number of genes' FROM genes;"

cursor.execute(query1)
result = cursor.fetchall()
result

[(6,)]

In [40]:
# to get the number we index the list and the tuple

result[0][0]

6

In [41]:
cursor.execute(query1)

print(get_header(cursor))
print(get_results(cursor))

Number of genes
6


In [42]:
# query2: How many GO terms we have?

query2 = "SELECT count(*) 'Number of GO terms' FROM GO_terms;"

cursor.execute(query2)

print(get_header(cursor))
print(get_results(cursor))

Number of GO terms
13


In [43]:
# query3: How many go terms are associated which each gene?   

query3 = '''
SELECT gene_id, count(GO_term_id) 'Number of GO terms' 
FROM gene_GO_term
GROUP BY gene_id;
'''

cursor.execute(query3)

print(get_header(cursor))
print(get_results(cursor))

gene_id	Number of GO terms
672	4
675	4
7040	2
7157	4
10296	3
64926	3


In [44]:
# if we want the gene_symbol

query3 = '''
SELECT gene_symbol Gene, count(gt.GO_term_id) 'Number of GO terms' 
FROM gene_GO_term gt
    JOIN genes g ON g.gene_id = gt.gene_id
GROUP BY gene_symbol;
'''

cursor.execute(query3)

print(get_header(cursor))
print(get_results(cursor))

Gene	Number of GO terms
BRCA1	4
BRCA2	4
MAEA	3
RASAL3	3
TGFB1	2
TP53	4


In [45]:
# query4: How many genes are associated with each GO term?  

query4 = '''
SELECT GO_term_id, count(gene_id) 'Number of genes' 
FROM gene_GO_term
GROUP BY GO_term_id;
'''

cursor.execute(query4)

print(get_header(cursor))
print(get_results(cursor))


GO_term_id	Number of genes
14	1
15	1
16	1
17	1
18	1
19	2
20	1
21	4
22	1
23	1
24	2
25	1
26	3


In [46]:
# if we want the GO term name

query4 = '''
SELECT name 'GO term', count(gene_id) 'Number of genes' 
FROM gene_GO_term gt
    JOIN GO_terms t ON t.GO_term_id = gt.GO_term_id
GROUP BY name;
'''

cursor.execute(query4)

# create a dataframe of the results
data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,GO term,Number of genes
0,cell adhesion,3
1,circadian behavior,1
2,histone acetyltransferase activity,1
3,intrinsic apoptotic signaling pathway in respo...,1
4,"negative regulation of transcription, DNA-temp...",1
5,positive regulation of transcription by RNA po...,1
6,promoter-specific chromatin binding,2
7,protease binding,2
8,protein binding,4
9,single-stranded DNA binding,1


In [47]:
# query5: Select all pairs of gene symbol and go term description that have a record in the gene_GO_term table.

query5 = '''
SELECT gene_symbol Gene, name 'GO term' 
FROM gene_GO_term gt
    JOIN GO_terms t ON t.GO_term_id = gt.GO_term_id
    JOIN genes g on g.gene_id = gt.gene_id;
'''

cursor.execute(query5)

print(get_header(cursor))
print(get_results(cursor))

Gene	GO term
BRCA1	intrinsic apoptotic signaling pathway in response to DNA damage
BRCA1	transcription cis-regulatory region binding
BRCA1	transcription coactivator activity
BRCA1	ubiquitin-protein transferase activity
BRCA2	histone acetyltransferase activity
BRCA2	protease binding
BRCA2	single-stranded DNA binding
BRCA2	protein binding
TP53	negative regulation of transcription, DNA-templated
TP53	positive regulation of transcription by RNA polymerase II
TP53	promoter-specific chromatin binding
TP53	circadian behavior
MAEA	cell adhesion
MAEA	protein binding
MAEA	promoter-specific chromatin binding
TGFB1	cell adhesion
TGFB1	protein binding
RASAL3	cell adhesion
RASAL3	protein binding
RASAL3	protease binding


In [48]:
# we are done working with this database
# close cursor and connection

cursor.close()
connection.close()

___

<b> <font color = "red">Exercise</font></b>

#### Query a database 
Download the database from the following link:
https://www.sqlitetutorial.net/sqlite-sample-database/

<img src = https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg width = 675>

https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg

Retrieve how many tracks are associated with each artist.    
What is the overall mean number of tracks per album?    
For a given artist, retrieve all the albums (name), and tracks (name), they are associated with.   

Feel free to build other quesries for questions you may have.


In [49]:
# Write your solution here

# connect to the database

connection = connect("chinook.db")
cursor = connection.cursor()


In [50]:
# check the tables in the database

sel_tables = "select name, type from sqlite_master" 
cursor.execute(sel_tables)

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,name,type
0,albums,table
1,sqlite_sequence,table
2,artists,table
3,customers,table
4,employees,table
5,genres,table
6,invoices,table
7,invoice_items,table
8,media_types,table
9,playlists,table


In [51]:
# task1: Retrieve how many tracks are associated with each artist.

task1 = '''
SELECT artistid, count(trackid) 
FROM tracks t 
   JOIN albums ab ON t.albumid = ab.albumid 
GROUP BY artistid
'''

cursor.execute(task1)

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,ArtistId,count(trackid)
0,1,18
1,2,4
2,3,15
3,4,13
4,5,12
...,...,...
199,271,1
200,272,1
201,273,1
202,274,1


In [52]:
# if we want the srtist name and ordered data

task1 = '''
SELECT  count(trackid) 'No of tracks', a.name
FROM tracks t 
   JOIN albums ab ON t.albumid = ab.albumid 
   JOIN artists a ON ab.artistid = a.artistid
GROUP BY a.name
ORDER BY count(trackid) DESC
'''

cursor.execute(task1)

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,No of tracks,Name
0,213,Iron Maiden
1,135,U2
2,114,Led Zeppelin
3,112,Metallica
4,92,Lost
...,...,...
199,1,"Academy of St. Martin in the Fields, Sir Nevil..."
200,1,"Academy of St. Martin in the Fields, John Birc..."
201,1,Academy of St. Martin in the Fields Chamber En...
202,1,Aaron Goldberg


In [53]:
# task2: What is the overall mean number of tracks per album?

# first let's get the number of Tracks per album before we do the mean
task2 = '''
SELECT count(trackid) 'Tracks per album'
FROM tracks t 
   JOIN albums ab ON t.albumid = ab.albumid 
GROUP BY ab.albumid
ORDER BY count(trackid) DESC
'''

cursor.execute(task2)

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,Tracks per album
0,57
1,34
2,30
3,26
4,25
...,...
342,1
343,1
344,1
345,1


Check the list of aggregate functions - we see that for mean we use avg()  
https://www.sqlite.org/lang_aggfunc.html

In [54]:
# let's do the mean of the results from the previous select 
# a select returns data in a table format so we can select from the result of a select

task2 = '''
SELECT avg(c) 'Mean number of tracks per album'
FROM
    (SELECT count(trackid) c
    FROM tracks t 
       JOIN albums ab ON t.albumid = ab.albumid 
    GROUP BY ab.albumid)
'''

cursor.execute(task2)

print(get_header(cursor))
print(get_results(cursor))

Mean number of tracks per album
10.095100864553315


In [55]:
# task3: For a given artist, retrieve all the albums (name), 
# and tracks (name), they are associated with.

task3 = '''
SELECT  a.name, title, t.name
FROM tracks t 
   JOIN albums ab ON t.albumid = ab.albumid 
   JOIN artists a ON ab.artistid = a.artistid
WHERE a.name = "Metallica"
ORDER BY title, t.name
'''

cursor.execute(task3)

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,Name,Title,Name
0,Metallica,...And Justice For All,...And Justice For All
1,Metallica,...And Justice For All,Blackened
2,Metallica,...And Justice For All,Dyers Eve
3,Metallica,...And Justice For All,Eye Of The Beholder
4,Metallica,...And Justice For All,Harvester Of Sorrow
...,...,...,...
107,Metallica,St. Anger,Shoot Me Again
108,Metallica,St. Anger,Some Kind Of Monster
109,Metallica,St. Anger,St. Anger
110,Metallica,St. Anger,Sweet Amber


In [56]:
# if we want to make a generic select where we supply the artist name
task3 = '''
SELECT  a.name, title, t.name
FROM tracks t 
   JOIN albums ab ON t.albumid = ab.albumid 
   JOIN artists a ON ab.artistid = a.artistid
WHERE a.name = ?
ORDER BY title, t.name
'''

cursor.execute(task3, ["AC/DC"])

data = cursor.fetchall()
header = [tp[0] for tp in cursor.description]
res_df = pd.DataFrame(data, columns = header)
res_df

,Name,Title,Name
0,AC/DC,For Those About To Rock We Salute You,Breaking The Rules
1,AC/DC,For Those About To Rock We Salute You,C.O.D.
2,AC/DC,For Those About To Rock We Salute You,Evil Walks
3,AC/DC,For Those About To Rock We Salute You,For Those About To Rock (We Salute You)
4,AC/DC,For Those About To Rock We Salute You,Inject The Venom
5,AC/DC,For Those About To Rock We Salute You,Let's Get It Up
6,AC/DC,For Those About To Rock We Salute You,Night Of The Long Knives
7,AC/DC,For Those About To Rock We Salute You,Put The Finger On You
8,AC/DC,For Those About To Rock We Salute You,Snowballed
9,AC/DC,For Those About To Rock We Salute You,Spellbound


In [57]:
# answer more interesting questions here

In [58]:
# when we are done working with the database
# close the cursor and the connection

cursor.close()
connection.close()


___

<b> <font color = "red">Exercise</font></b>

#### Variant database

Create a variant sqlite database with 2 tables variant and var_info from the file `variant_file.vcf`.    
The header of the file is:
`#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO	FORMAT	Group_A/IP2-50/tophat_out/IP2-50_accepted_hits.bam`

The first 6 columns from the file will be columns in the `variant` table. The `variant` table will also have a `variant_id` - autoincrement column to uniquely identify the variant in this database.    


The `var_info` table will contain the data from the INFO column in the file.  The `var_info` table will have the following columns: `variant_id`, `tag` and `value`. 
For each variant, the pairs tag=values are separated by ";" and the values are separated by ",".  Each entry in the var_info table will have a unique variant_id, tag and value column combination.


Run the following querries:
- A select that retrieves how many variants we have.
- A select of the variant(s) with the highest number associated tag-value pairs.
- A select of INDEL variants with DP values > 150.




In [59]:
# Write your solution here

# create and connect to the variant database
# this should be a new file
# make sure it does not exist in your current folder

connection = connect("variant.sqlite")

# create a cursor
cursor = connection.cursor()


In [60]:
# create tables in the database

# write the create statements

#-------
create_variant = '''
CREATE TABLE IF NOT EXISTS variant (
      variant_id INTEGER PRIMARY KEY AUTOINCREMENT,
      CHROM TEXT NOT NULL,
      POS BIGINT  NOT NULL,
      ID TEXT,
      REF TEXT, 
      ALT TEXT, 
      QUAL TEXT
    );
'''
#-------
create_var_info = '''
CREATE TABLE IF NOT EXISTS var_info (
      variant_id INTEGER NOT NULL,
      tag INTEGER NOT NULL,
      value INTEGER NOT NULL
    );
'''
#-------

In [61]:
# we do the same thing multiple times - we can use a function

def run_create_table(sql, cursor, connection):
    try:
        cursor.execute(sql)
    except connection.DatabaseError:
        print("Creating the table resulted in a database error!")
        connection.rollback()
        raise
    else:
        connection.commit()
    finally:
        print("done!")

In [62]:
# check the sqlite_master table - should be empty at this point

select_master = "SELECT name, type FROM sqlite_master;"
cursor.execute(select_master)
cursor.fetchall()

[]

In [63]:
# run the commands
run_create_table(create_variant, cursor, connection)
run_create_table(create_var_info, cursor, connection)


done!
done!


In [64]:
# check the sqlite_master table 
# should have the two tables

select_master = "SELECT name, type FROM sqlite_master;"
cursor.execute(select_master)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,name,type
0,variant,table
1,sqlite_sequence,table
2,var_info,table


In [65]:
# write the insert statements

#-------
insert_variant = """
INSERT INTO variant (CHROM, POS, ID, REF, ALT, QUAL) VALUES (?,?,?,?,?,?)
"""
#-------

insert_info = """
INSERT INTO var_info VALUES (?,?,?)
"""
#-------


In [66]:
# parse the file and add data to the two tables

# open file for reading
with open("variant_file.vcf") as vf:
    # go through file lines
    for line in vf:
        # skip comment lines
        if not line.startswith("#"):
            # remove new line at the end and break the line by tab
            line_lst = line.strip().split("\t")
            # take the first 6 elements - to go into a variant table row
            var_vals = line_lst[:6]
            # run the insert command
            cursor.execute(insert_variant, var_vals)
            # take the variant id from the cursor attribute lastrowid
            variant_id = cursor.lastrowid
            # take the info column data 3rd from the end 8th from the beginning
            # and break it by ";"
            info_lst = line_lst[-3].split(";")
            # go though the list - it conains tag=values pairs
            for pair in info_lst:
                # break the pair by =
                pair_lst = pair.split("=")
                # if the list has only one element 
                # add that as a tag with value 1 (True)
                if len(pair_lst) == 1:
                    var_info_vals = [variant_id, pair, 1]
                    # execute insert for variant info
                    cursor.execute(insert_info, var_info_vals)
                else:
                    # separate tag and values 
                    # split the vals by ","
                    tag, vals = pair_lst
                    for val in vals.split(","):
                        # execute insert for variant info
                        # create the list of three values
                        # use eval to evaluate the value to the proper datatype
                        # i.e. make the val string a number 
                        var_info_vals = [variant_id, tag, eval(val)]
                        # execute insert for variant info
                        cursor.execute(insert_info, var_info_vals)
                        
    # commit/save the inserts/changes
    connection.commit()
                        

            
            
            
            
            

In [67]:
# check the variant table 
# should have data now

select_variant = "SELECT * FROM variant;"
cursor.execute(select_variant)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,CHROM,POS,ID,REF,ALT,QUAL
0,1,1,237763,.,G,A,6.02
1,2,1,564452,.,A,G,19.6
2,3,1,564464,.,GGG,GGGG,20
3,4,1,565286,.,C,T,36.1
4,5,1,567242,.,G,A,18.8
...,...,...,...,...,...,...,...
193604,193605,Y,59025774,.,C,T,17.1
193605,193606,Y,59025960,.,C,G,6.02
193606,193607,Y,59027700,.,A,C,30.1
193607,193608,Y,59027910,.,G,A,5.29


In [68]:
# check the var_info table 
# should have data now

select_var_info = "SELECT * FROM var_info;"
cursor.execute(select_var_info)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,tag,value
0,1,DP,2.0000
1,1,VDB,0.0261
2,1,AF1,1.0000
3,1,AC1,2.0000
4,1,DP4,0.0000
...,...,...,...
2034307,193609,DP4,0.0000
2034308,193609,DP4,2.0000
2034309,193609,DP4,0.0000
2034310,193609,MQ,20.0000


In [69]:
# Run the querries

In [70]:
# A select that retrieves how many variants we have.

# we use count

select_var_count = "SELECT count(variant_id) 'Var. No.' FROM variant;"
cursor.execute(select_var_count)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res



,Var. No.
0,193609


In [71]:
# A select of the variant(s) with the highest number associated tag-value pairs.


# count the number of tag-val pairs per variant 
# we use group by to group by variant_id in the var_info table
# use order by to sort the data

select_var_infocount = """
SELECT variant_id, count(*) InfoNo 
FROM var_info
GROUP BY variant_id
ORDER BY count(*)  DESC;
"""
cursor.execute(select_var_infocount)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res


,variant_id,InfoNo
0,193584,15
1,193574,15
2,193513,15
3,193313,15
4,193303,15
...,...,...
193604,9,10
193605,8,10
193606,4,10
193607,2,10


In [72]:
# count the number of tag-val pairs per variant 
# we use group by to group by variant_id in the var_info table
# use order by to sort the data

# select the top count - order by desc and limit 1

select_var_infocount = """
SELECT InfoNo
FROM 
    (SELECT variant_id, count(*) InfoNo 
    FROM var_info
    GROUP BY variant_id
    ORDER BY count(*) DESC  
    LIMIT 1);
"""
cursor.execute(select_var_infocount)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res


,InfoNo
0,15


In [73]:
# count the number of tag-val pairs per variant 
# we use group by to group by variant_id in the var_info table
# use order by to sort the data

# select the max count 

select_var_infocount = """
SELECT max(InfoNo)
FROM 
    (SELECT variant_id, count(*) InfoNo 
    FROM var_info
    GROUP BY variant_id
    ORDER BY count(*) DESC);
"""
cursor.execute(select_var_infocount)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,max(InfoNo)
0,15


In [74]:
# count the number of tag-val pairs per variant 
# we use group by to group by variant_id in the var_info table
# use order by to sort the data

# select the max count
# SELECT the variants with the max count using a join to get variant data 
# from the variant table and count for the query that does the count
# then use having to only select variant with the count equal to the max

select_var_infocount = """
SELECT *
FROM variant v INNER JOIN 
    (SELECT variant_id, count(*) InfoNo 
    FROM var_info
    GROUP BY variant_id  
    HAVING InfoNo IN (SELECT InfoNo FROM 
                        (SELECT variant_id, count(*) InfoNo 
                         FROM var_info
                         GROUP BY variant_id
                         ORDER BY count(*) DESC  
                         LIMIT 1))
    ORDER BY count(*) DESC
    ) s ON s.variant_id = v.variant_id 
"""
cursor.execute(select_var_infocount)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,CHROM,POS,ID,REF,ALT,QUAL,variant_id,InfoNo
0,193584,Y,59009514,.,GC,G,21.5,193584,15
1,193574,Y,59001398,.,G,GGCGG,3.43,193574,15
2,193513,Y,21152664,.,AAA,A,123,193513,15
3,193313,X,153590944,.,CCCC,CCCCC,4.07,193313,15
4,193303,X,153577133,.,T,TAG,117,193303,15
...,...,...,...,...,...,...,...,...,...
1740,1214,1,17393706,.,AAAGAA,AAA,41.8,1214,15
1741,962,1,14276752,.,GCC,GC,8.73,962,15
1742,946,1,14106397,.,CCT,CCTCCT,53.9,946,15
1743,225,1,3697549,.,A,"AA,AAA",24.9,225,15


In [75]:
# A select of the DP values for INDEL variants.


# select indel variants 
select_indel = """
SELECT variant_id, tag, value
FROM var_info
WHERE tag = 'INDEL';
"""


cursor.execute(select_indel)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,tag,value
0,3,INDEL,1
1,6,INDEL,1
2,16,INDEL,1
3,30,INDEL,1
4,86,INDEL,1
...,...,...,...
8633,193545,INDEL,1
8634,193568,INDEL,1
8635,193574,INDEL,1
8636,193584,INDEL,1


In [76]:
# A select of INDEL variants with DP values > 150.


# select variant data from variant table and 
# and take only the variants that are indels
# inner join with the var_info table to get the dp values  


select_indel = """
SELECT *
FROM variant 
    INNER JOIN var_info ON variant.variant_id = var_info.variant_id
WHERE
    tag = 'DP' AND
    value > 150 AND
    variant.variant_id IN 
        (SELECT variant_id
        FROM var_info
        WHERE tag = 'INDEL');
"""


cursor.execute(select_indel)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,CHROM,POS,ID,REF,ALT,QUAL,variant_id,tag,value
0,3305,1,40537941,.,AAA,ACAAA,23.7,3305,DP,158
1,4555,1,59096779,.,GA,GAA,140,4555,DP,163
2,18901,10,5807742,.,A,"AA,AAA",53.5,18901,DP,180
3,25814,10,102123890,.,T,TT,12.7,25814,DP,153
4,35801,11,130014581,.,TTT,TT,90.5,35801,DP,159
5,38296,12,19566184,.,GTTT,GTT,88.5,38296,DP,190
6,40640,12,53435294,.,TTTTTTT,TTTTTTTTTT,150,40640,DP,162
7,53399,14,50329569,.,GT,"GAAT,GCT",44.5,53399,DP,204
8,65374,16,33964064,.,T,TC,202,65374,DP,179
9,81965,19,13054921,.,T,TT,20.5,81965,DP,151


In [77]:
# A select of variants with rsid

select_rs = """
SELECT *
FROM variant 
WHERE ID LIKE 'rs%';
"""


cursor.execute(select_rs)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res



,variant_id,CHROM,POS,ID,REF,ALT,QUAL


In [78]:
# A select of variants with id not dot

select_rs = """
SELECT *
FROM variant 
WHERE ID <> ".";
"""


cursor.execute(select_rs)

data = cursor.fetchall()
header = [ct[0] for ct in cursor.description]
df_res = pd.DataFrame(data, columns = header)
df_res

,variant_id,CHROM,POS,ID,REF,ALT,QUAL


In [79]:
# we are done working with this database
# close cursor and connection

cursor.close()
connection.close()